In [ ]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False


In [ ]:
%sql duckdb:///:default:

In [ ]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('~/buckets/b1/datasets/competencia_02_preFE.csv.gz')

In [ ]:
%%sql
CREATE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

In [ ]:
%%sql
CREATE MACRO calculo_ratio(a, b) AS NULLIF(a / NULLIF(b,0),0);

In [ ]:
%%sql
CREATE MACRO corrijo_drift_rank(a,b) AS percent_rank() over (partition by b order by a);

#### Aplicando funciones para Competencia02

In [ ]:
%%sql
CREATE TABLE table_1 AS
SELECT
    *,
    
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as tc_transacciones_total,
    suma_sin_null(ctarjeta_visa, ctarjeta_master) as tc_ctarjeta_total,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as tc_consumo_total,
    suma_sin_null(Master_mconsumototal, Visa_mconsumototal) as tc_mconsumo_total,
    suma_sin_null(Visa_mlimitecompra, Master_mlimitecompra) as tc_limcompra_total,
    suma_sin_null(Master_mpagado, Visa_mpagado) as tc_pagado_total,
    suma_sin_null(Master_mpagominimo, Visa_mpagominimo) as tc_saldo_total,
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_pagomin_total,
    ifnull(mprestamos_personales, 0) + ifnull(mprestamos_prendarios, 0) + ifnull(mprestamos_hipotecarios, 0) as pasivos_total,
    ifnull(mplazo_fijo_dolares, 0) + ifnull(mplazo_fijo_pesos, 0) + ifnull(minversion1_pesos, 0) 
        + ifnull(minversion1_dolares, 0) + ifnull(minversion2, 0) as activos_total,
    ifnull(mpayroll, 0) + ifnull(mpayroll2, 0) as flujos_ingresos_total,
    ifnull(mcajeros_propios_descuentos, 0) + ifnull(mtarjeta_visa_descuentos, 0) + ifnull(mtarjeta_master_descuentos, 0) as beneficios_total,
    ifnull(mcomisiones_mantenimiento, 0) + ifnull(mcomisiones_otras, 0) as comisiones_total,
    ifnull(mcuenta_debitos_automaticos, 0) + ifnull(mttarjeta_visa_debitos_automaticos, 0) + ifnull(mttarjeta_master_debitos_automaticos, 0) 
        + ifnull(mpagodeservicios, 0) + ifnull(mpagomiscuentas, 0) as gastos_total,


    calculo_ratio(tc_consumo_total, flujos_ingresos_total) as consumo_sobre_ingresos,
    calculo_ratio(tc_mconsumo_total, flujos_ingresos_total) as tcm_consumo_sobre_ingresos,
    calculo_ratio(tc_pagado_total, flujos_ingresos_total) as pagado_sobre_ingresos,
    calculo_ratio(tc_pagado_total, activos_total) as pagado_sobre_activos,
    calculo_ratio(tc_pagomin_total, flujos_ingresos_total) as pagomin_sobre_ingresos,
    calculo_ratio(tc_consumo_total, activos_total) as gastos_sobre_activos,
    calculo_ratio(activos_total, pasivos_total) as activos_sobre_pasivos,
    calculo_ratio(beneficios_total, comisiones_total) as beneficios_sobre_comisiones,
    calculo_ratio(beneficios_total, tc_consumo_total) as beneficios_sobre_consumos,
    calculo_ratio(gastos_total, flujos_ingresos_total) as gastos_sobre_ingresos,
    calculo_ratio(tc_limcompra_total, tc_consumo_total) as lim_sobre_consumos,
    

    corrijo_drift_rank(tc_mconsumo_total, foto_mes) as tc_mconsumo_total_sin_drift,
    corrijo_drift_rank(tc_limcompra_total, foto_mes) as tc_limcompra_total_sin_drift,
    corrijo_drift_rank(tc_pagado_total, foto_mes) as tc_pagado_total_sin_drift,
    corrijo_drift_rank(pasivos_total, foto_mes) as pasivos_total_sin_drift,
    corrijo_drift_rank(activos_total, foto_mes) as activos_total_sin_drift,
    corrijo_drift_rank(flujos_ingresos_total, foto_mes) as flujos_ingresos_total_sin_drift,
    corrijo_drift_rank(comisiones_total, foto_mes) as comisiones_total_sin_drift,
    corrijo_drift_rank(gastos_total, foto_mes) as gastos_total_sin_drift,
    
   
     
FROM competencia_01

In [ ]:
%%sql
DROP TABLE IF EXISTS competencia_01

In [ ]:
%%sql
CREATE TABLE table_2 AS
SELECT
    *,

    lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_ctrx_quarter_sin_drift,
    lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_ctrx_quarter_sin_drift,
    lag(ctrx_quarter, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_ctrx_quarter_sin_drift,

    lag(tc_mconsumo_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_tc_mconsumo_total_sin_drift,
    lag(tc_mconsumo_total_sin_drift, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_tc_mconsumo_total_sin_drift,
    lag(tc_mconsumo_total_sin_drift, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_tc_mconsumo_total_sin_drift,

    lag(tc_limcompra_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_tc_limcompra_total_sin_drift,
    lag(tc_limcompra_total_sin_drift, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_tc_limcompra_total_sin_drift,
    lag(tc_limcompra_total_sin_drift, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_tc_limcompra_total_sin_drift,

    lag(tc_pagado_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_tc_pagado_total_sin_drift,
    lag(tc_pagado_total_sin_drift, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_tc_pagado_total_sin_drift,
    lag(tc_pagado_total_sin_drift, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_tc_pagado_total_sin_drift,

    lag(pasivos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_pasivos_total_sin_drift,
    lag(pasivos_total_sin_drift, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_pasivos_total_sin_drift,
    lag(pasivos_total_sin_drift, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_pasivos_total_sin_drift,

    lag(activos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_activos_total_sin_drift,
    lag(activos_total_sin_drift, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_activos_total_sin_drift,
    lag(activos_total_sin_drift, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_activos_total_sin_drift,

    lag(flujos_ingresos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_flujos_ingresos_tot_sin_drift,
    lag(flujos_ingresos_total_sin_drift, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_flujos_ingresos_tot_sin_drift,
    lag(flujos_ingresos_total_sin_drift, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_flujos_ingresos_tot_sin_drift,

    lag(comisiones_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_comisiones_tot_sin_drift,
    lag(comisiones_total_sin_drift, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_comisiones_tot_sin_drift,
    lag(comisiones_total_sin_drift, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_comisiones_tot_sin_drift,

    lag(gastos_total_sin_drift, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_gastos_tot_sin_drift,
    lag(gastos_total_sin_drift, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_gastos_tot_sin_drift,
    lag(gastos_total_sin_drift, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_gastos_tot_sin_drift,

    lag(internet, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_internet,
    lag(internet, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_internet,
    lag(internet, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_internet,
    
    lag(cdescubierto_preacordado, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_cdescubierto_preacordado,
    lag(cdescubierto_preacordado, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_cdescubierto_preacordado,
    lag(cdescubierto_preacordado, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_cdescubierto_preacordado,

    lag(cproductos, 1) over (partition by numero_de_cliente order by foto_mes) as lag1_cproductos,
    lag(cproductos, 3) over (partition by numero_de_cliente order by foto_mes) as lag3_cproductos,
    lag(cproductos, 6) over (partition by numero_de_cliente order by foto_mes) as lag6_cproductos,
    
     
FROM table_1

In [ ]:
%%sql
DROP TABLE IF EXISTS table_1

In [ ]:
%%sql
CREATE TABLE table_3 AS
SELECT
    *,
        
    avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media6, 
    max(ctrx_quarter) over ventana_6 as ctrx_quarter_max6, 
    min(ctrx_quarter) over ventana_6 as ctrx_quarter_min6,
    regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope6,
    
    avg(cproductos) over ventana_6 as cproductos_media6, 
    max(cproductos) over ventana_6 as cproductos_max6, 
    min(cproductos) over ventana_6 as cproductos_min6,
    regr_slope(cproductos, cliente_antiguedad) over ventana_6 as cproductos_slope6,
    
    avg(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_tr_media6, 
    max(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_tr_max6, 
    min(ctarjeta_debito_transacciones) over ventana_6 as ctarjeta_debito_tr_min6,
    regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_debito_tr_slope6,

    avg(chomebanking_transacciones) over ventana_6 as chomebanking_tr_media6, 
    max(chomebanking_transacciones) over ventana_6 as chomebanking_tr_max6, 
    min(chomebanking_transacciones) over ventana_6 as chomebanking_tr_min6,
    regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_6 as chomebanking_tr_slope6,

    avg(tc_transacciones_total) over ventana_6 as tc_transacciones_total_media6, 
    max(tc_transacciones_total) over ventana_6 as tc_transacciones_total_max6, 
    min(tc_transacciones_total) over ventana_6 as tc_transacciones_total_min6,
    regr_slope(tc_transacciones_total, cliente_antiguedad) over ventana_6 as tc_transacciones_total_slope6,    
    
    avg(cpayroll_trx) over ventana_6 as cpayroll_trx_media6, 
    max(cpayroll_trx) over ventana_6 as cpayroll_trx_max6, 
    min(cpayroll_trx) over ventana_6 as cpayroll_trx_min6,
    regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_6 as cpayroll_trx_slope6,   
    
    avg(ctransferencias_recibidas) over ventana_6 as ctransferencias_recibidas_media6, 
    max(ctransferencias_recibidas) over ventana_6 as ctransferencias_recibidas_max6, 
    min(ctransferencias_recibidas) over ventana_6 as ctransferencias_recibidas_min6,
    regr_slope(ctransferencias_recibidas, cliente_antiguedad) over ventana_6 as ctransferencias_recibidas_slope6,      
    
    avg(ctransferencias_emitidas) over ventana_6 as ctransferencias_emitidas_media6, 
    max(ctransferencias_emitidas) over ventana_6 as ctransferencias_emitidas_max6, 
    min(ctransferencias_emitidas) over ventana_6 as ctransferencias_emitidas_min6,
    regr_slope(ctransferencias_emitidas, cliente_antiguedad) over ventana_6 as ctransferencias_emitidas_slope6,    
    
    avg(chomebanking_transacciones) over ventana_6 as chomebanking_tr_media6, 
    max(chomebanking_transacciones) over ventana_6 as chomebanking_tr_max6, 
    min(chomebanking_transacciones) over ventana_6 as chomebanking_tr_min6,
    regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_6 as chomebanking_tr_slope6,    
     
FROM table_2
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)

In [ ]:
%%sql
DROP TABLE IF EXISTS table_2

In [1]:
%%sql
ALTER TABLE table_3 DROP COLUMN ctarjeta_visa_transacciones;
ALTER TABLE table_3 DROP COLUMN ctarjeta_master_transacciones;
ALTER TABLE table_3 DROP COLUMN ctarjeta_visa;
ALTER TABLE table_3 DROP COLUMN ctarjeta_master;
ALTER TABLE table_3 DROP COLUMN mtarjeta_visa_consumo;
ALTER TABLE table_3 DROP COLUMN mtarjeta_master_consumo;
ALTER TABLE table_3 DROP COLUMN Master_mconsumototal;
ALTER TABLE table_3 DROP COLUMN Visa_mconsumototal;
ALTER TABLE table_3 DROP COLUMN Visa_mlimitecompra;
ALTER TABLE table_3 DROP COLUMN Master_mlimitecompra;
ALTER TABLE table_3 DROP COLUMN Master_mpagado;
ALTER TABLE table_3 DROP COLUMN Visa_mpagado;
ALTER TABLE table_3 DROP COLUMN Master_mpagominimo;
ALTER TABLE table_3 DROP COLUMN Visa_mpagominimo;
ALTER TABLE table_3 DROP COLUMN Master_msaldototal;
ALTER TABLE table_3 DROP COLUMN Visa_msaldototal;
ALTER TABLE table_3 DROP COLUMN mprestamos_personales;
ALTER TABLE table_3 DROP COLUMN mprestamos_prendarios;
ALTER TABLE table_3 DROP COLUMN mprestamos_hipotecarios;
ALTER TABLE table_3 DROP COLUMN mplazo_fijo_pesos;
ALTER TABLE table_3 DROP COLUMN minversion1_pesos;
ALTER TABLE table_3 DROP COLUMN minversion1_dolares;
ALTER TABLE table_3 DROP COLUMN minversion2;
ALTER TABLE table_3 DROP COLUMN mpayroll;
ALTER TABLE table_3 DROP COLUMN mpayroll2;
ALTER TABLE table_3 DROP COLUMN mcajeros_propios_descuentos;
ALTER TABLE table_3 DROP COLUMN mtarjeta_visa_descuentos;
ALTER TABLE table_3 DROP COLUMN mtarjeta_master_descuentos;
ALTER TABLE table_3 DROP COLUMN mcomisiones_mantenimiento;
ALTER TABLE table_3 DROP COLUMN mcomisiones_otras;
ALTER TABLE table_3 DROP COLUMN mcuenta_debitos_automaticos;
ALTER TABLE table_3 DROP COLUMN mttarjeta_visa_debitos_automaticos;
ALTER TABLE table_3 DROP COLUMN mttarjeta_master_debitos_automaticos;
ALTER TABLE table_3 DROP COLUMN mpagodeservicios;
ALTER TABLE table_3 DROP COLUMN mpagomiscuentas;
ALTER TABLE table_3 DROP COLUMN tc_mconsumo_total;
ALTER TABLE table_3 DROP COLUMN tc_limcompra_total;
ALTER TABLE table_3 DROP COLUMN tc_pagado_total;
ALTER TABLE table_3 DROP COLUMN pasivos_total;
ALTER TABLE table_3 DROP COLUMN activos_total;
ALTER TABLE table_3 DROP COLUMN flujos_ingresos_total;
ALTER TABLE table_3 DROP COLUMN comisiones_total;
ALTER TABLE table_3 DROP COLUMN gastos_total;



UsageError: Cell magic `%%sql` not found.


In [ ]:
%%sql
copy table_3 to '~/buckets/b1/datasets/competencia_02_v01.csv.gz' (FORMAT CSV, HEADER)